In [2]:
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

import numpy as np
import json
import os
import shutil
import unicodedata
import uuid
import cv2
import mediapipe as mp

/home/ant/miniconda3/envs/psl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def create_new_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)           # Removes all the subdirectories!
        print('Directory existed and was cleaned')
        os.makedirs(path)

In [6]:
folder_path_raw = '../dataset/images_raw'
folder_path = '../dataset/images'

In [7]:
def convert_to_jpg(file_path, path_to_save):
    img = Image.open(file_path)
    if img.format == 'PNG': # Conversion of png to jpg
        file_name = os.path.splitext(path_to_save)[0]
        img = img.convert("RGB")
        img.save(file_name + '.jpg', 'JPEG')
    else:
        file_name = os.path.splitext(path_to_save)[0]
        img.save(file_name + '.jpg', 'JPEG')

create_new_folder(folder_path)

for file_name in tqdm(os.listdir(folder_path_raw)):
    if file_name.lower().endswith('.png') or  file_name.lower().endswith('.jpg'):
        file_path = os.path.join(folder_path_raw, file_name)
        path_to_save = os.path.join(folder_path, file_name)
        convert_to_jpg(file_path, path_to_save)

Directory existed and was cleaned


100%|██████████| 3741/3741 [02:08<00:00, 29.04it/s]


In [8]:
with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
    print(len(list(entries)))

3702


In [9]:
def extract_labels(folder_path):
    labels = {}
    
    skipped = 0
    created = 0
    labels_folder = '../dataset/labels/'
    images_path = '../dataset/images_renamed'
    
    create_new_folder(labels_folder)
    create_new_folder(images_path)

    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True)
    mp_drawing = mp.solutions.drawing_utils

    # Searching for files in a folder
    with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
        for entry in tqdm(entries):
            if entry.is_file() and entry.name.lower().endswith('.jpg'): # I want to exclude files that are not jpg
                filename = entry.name
                #print(filename)
                # Extract label from the file name, ignoring dashes
                label = ''.join([c for c in filename.split('.')[0] if not (c.isdigit() or c == '-')])
                
                # Skip file names starting with 'img'
                if filename.lower().startswith('img'):
                    # print(f'skipping starting with img {filename}')
                    skipped += 1
                    continue
                
                # Normalize the filename to ensure proper handling of diacritical marks
                normalized_filename = unicodedata.normalize('NFC', filename)

                # Check if filename contains 'sz', 'cz', 'ch' or 'rz'
                if 'sz' in filename:
                    labels[filename] = 'sz'
                elif 'cz' in filename:
                    labels[filename] = 'cz'
                elif 'ch' in filename:
                    labels[filename] = 'ch'
                elif 'rz' in filename:
                    labels[filename] = 'rz'
                else:    
                    # Retrieving the first letter of the file name    
                    labels[normalized_filename] = label
                
                # Prepare the label dictionary for the current image
                image_labels = {'label': label}
                
                new_file_name = str(uuid.uuid4()).replace('-', '')
                shutil.copyfile(os.path.join(folder_path, filename), os.path.join(images_path, new_file_name + '.jpg'))

                # Process image using MediaPipe: loading image
                image = cv2.imread(os.path.join(folder_path, filename))  # Loading the image from disk
                # Convert the image to RGB format (MediaPipe requires RGB input)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Processing the image using MediaPipe Hands
                results = hands.process(image_rgb)

                # Get landmarks and add them to the label dictionary
                if results.multi_hand_landmarks:  # Checking if hand landmarks exist in the image processing results
                    for hand_landmarks in results.multi_hand_landmarks:
                        landmarks = hand_landmarks.landmark  # Retrieving the hand landmarks
                        
                        # Creating a dictionary containing the coordinates of the hand landmarks
                        landmark_dict = {f'hand_landmark_{i}': {'x': landmark.x, 'y': landmark.y, 'z': landmark.z} for i, landmark in enumerate(landmarks)}
                        image_labels['hand_landmarks'] = landmark_dict  # Adding the hand landmarks to the label dictionary for the image
                
               # Saving the results to a JSON file
                with open(os.path.join(labels_folder, f'{new_file_name}.json'), 'w', encoding='utf-8') as json_file:
                    json.dump(image_labels, json_file,  indent=4, ensure_ascii=False)
                created += 1
            else:
                skipped += 1  
    print(created, skipped)

# Calling the function
extract_labels(folder_path)

Directory existed and was cleaned
Directory existed and was cleaned


0it [00:00, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
3702it [02:17, 26.96it/s]

3626 76


In [10]:
def count_files_with_landmarks(labels_folder):
    labeled_with_landmarks_count = 0
    labeled_without_landmarks_count = 0
    data_rows = []
    # Iterate through files in the labels folder
    for filename in tqdm(os.listdir(labels_folder)):
        if filename.endswith('.json'):  # Check if the file is a JSON file
            with open(os.path.join(labels_folder, filename), 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                # Check if the JSON file contains the 'landmarks' section
                if 'hand_landmarks' in data:
                    labeled_with_landmarks_count += 1

                    # Inspect the structure of 'hand_landmarks'
                    landmarks_data = data['hand_landmarks']
                    # print(landmarks_data)  # Print or inspect the structure
                        
                    # Initialisation of the line for each characteristic point
                    row = []
                    
                    # Adding x, y, z coordinates of each point to the row
                    for landmark_key in landmarks_data:
                        landmark = landmarks_data[landmark_key]
                        row.extend([landmark['x'], landmark['y'], landmark['z']])
                    
                    # Adding a label at the end of the line
                    row.append(data['label'])  # Use the label as a label
                    
                    # Adding a row to the data list
                    data_rows.append(row)

                    # print(data_rows[0])  # Display of the first line for the example

                else:
                    labeled_without_landmarks_count += 1
    print(data_rows[0])
    print(len(data_rows[0]))
    return labeled_with_landmarks_count, labeled_without_landmarks_count

In [11]:
labels_path= '../dataset/labels'
number_of_labeled_files_with_landmarks, number_of_labeled_files_without_landmarks = count_files_with_landmarks(labels_path)
print(f'Number of JSON files with landmarks information: {number_of_labeled_files_with_landmarks}')
print(f'Number of JSON files without landmarks information: {number_of_labeled_files_without_landmarks}')

100%|██████████| 3626/3626 [00:00<00:00, 20249.04it/s]

[0.5722715258598328, 0.9279959201812744, 6.085367658670293e-07, 0.6667962670326233, 0.8835490942001343, -0.05172787234187126, 0.7500747442245483, 0.7784339189529419, -0.08351358026266098, 0.8113635182380676, 0.7292087078094482, -0.11344535648822784, 0.8760277032852173, 0.6948061585426331, -0.1403423249721527, 0.6881923675537109, 0.5869115591049194, -0.053579822182655334, 0.7228823304176331, 0.4431452751159668, -0.1276901513338089, 0.7501876950263977, 0.3440498411655426, -0.16761082410812378, 0.755721926689148, 0.2457873523235321, -0.19032491743564606, 0.6012634634971619, 0.5760300755500793, -0.06292890012264252, 0.6995595097541809, 0.5268629789352417, -0.1440703272819519, 0.7008561491966248, 0.6791463494300842, -0.14395952224731445, 0.6560532450675964, 0.7099243998527527, -0.12153636664152145, 0.5214959979057312, 0.6006360054016113, -0.08299653977155685, 0.6147222518920898, 0.5985482931137085, -0.1846313327550888, 0.6262587904930115, 0.7452564239501953, -0.16008451581001282, 0.58300048